# 🎛️ Professional Mixing Session
## Multi-Channel Mixing → Stem Export → Post-Mix Mastering Pipeline

This notebook provides professional mixing of raw audio channels, creating perfectly balanced stems that feed into your existing post-mix mastering pipeline.

**Workflow:**
1. Load raw channels (organized by instrument groups)
2. Apply professional mixing (EQ, compression, spatial positioning, effects)
3. Export mixed stems (drums, bass, vocals, music)
4. Feed into existing post-mix mastering pipeline


In [1]:
# Core imports and setup
import os
import json
import numpy as np
import soundfile as sf
from pathlib import Path
from typing import Dict, List, Tuple, Optional
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Import our mixing and mastering systems
from mixing_engine import MixingSession, ChannelStrip, MixBus
from channel_recognition import identify_channel_type, suggest_processing
from mix_templates import MixTemplate, get_template

print("🎛️ Professional Mixing System Ready!")
print("📁 Supports multi-channel input with intelligent processing")
print("🎚️ Outputs professional stems for mastering pipeline")

DSP Primitives Layer loaded:
- Gain/level: apply_gain_db, normalize_peak, normalize_lufs, measure_peak, measure_rms
- Filters: highpass_filter, lowpass_filter, bandpass_filter, shelf_filter, peaking_eq, notch_filter, tilt_eq
- Stereo: mid_side_encode, mid_side_decode, stereo_widener
- Dynamics: compressor (soft‑knee), transient_shaper
- Fades: fade_in, fade_out
- K‑weighting/LUFS approx: k_weight, lufs_integrated_approx
🎛️ Mixing Engine loaded!
   • Multi-channel support with intelligent routing
   • Template-based processing
   • Bus architecture with grouping
   • Exports stems for post-mix pipeline
🔍 Channel Recognition System loaded!
   • Name pattern matching with 40+ instrument types
   • Frequency analysis for content identification
   • Automatic processing suggestions
   • User hint support for ambiguous channels
🎨 Mix Templates loaded!
   • 5 genre-specific templates (Pop, Rock, EDM, Hip-Hop, Jazz)
   • Channel-specific EQ, compression, and effects
   • Intelligent spatial po

## 📁 Step 1: Define Your Input Channels

Organize your raw audio files by instrument category. The system will intelligently process each based on:
- Channel name (kick, snare, bass, etc.)
- Category placement (drums, bass, vocals, etc.)
- Frequency analysis (auto-detection)
- Mix template (genre-specific processing)

In [2]:
# # OPTION 1: Simple channel definition (modify paths to your files)
# channels = {
#     "drums": {
#         "kick": "/path/to/kick.wav",
#         "snare": "/path/to/snare.wav",
#         "hihat": "/path/to/hihat.wav",
#         "toms": "/path/to/toms.wav",
#         "overhead_L": "/path/to/overhead_L.wav",
#         "overhead_R": "/path/to/overhead_R.wav",
#     },
#     "bass": {
#         "bass_di": "/path/to/bass_di.wav",
#         "bass_amp": "/path/to/bass_amp.wav",
#     },
#     "guitars": {
#         "rhythm_L": "/path/to/rhythm_gtr_L.wav",
#         "rhythm_R": "/path/to/rhythm_gtr_R.wav",
#         "lead": "/path/to/lead_guitar.wav",
#     },
#     "keys": {
#         "piano": "/path/to/piano.wav",
#         "synth_pad": "/path/to/pad.wav",
#         "synth_lead": "/path/to/lead_synth.wav",
#     },
#     "vocals": {
#         "lead": "/path/to/lead_vocal.wav",
#         "double": "/path/to/vocal_double.wav",
#         "harmony_1": "/path/to/harmony_1.wav",
#         "harmony_2": "/path/to/harmony_2.wav",
#         "adlibs": "/path/to/adlibs.wav",
#     },
#     "other": {
#         # For ambiguous or unnamed elements
#         "element_1": "/path/to/unknown_synth.wav",
#         "element_2": "/path/to/texture.wav",
#     }
# }
#
# # OPTION 2: Advanced channel definition with hints
# channels_advanced = {
#     "drums": {
#         "kick": {
#             "path": "/path/to/kick.wav",
#             "hints": {"subtype": "electronic", "mic": "close"}  # Optional hints
#         },
#         "snare": {
#             "path": "/path/to/snare.wav",
#             "hints": {"subtype": "acoustic", "mic": "top"}
#         },
#     },
#     "synths": {  # Custom category
#         "wobble_bass": {
#             "path": "/path/to/wobble.wav",
#             "hints": {"role": "bass", "style": "dubstep"}
#         },
#         "pluck_lead": {
#             "path": "/path/to/pluck.wav",
#             "hints": {"role": "lead", "style": "edm"}
#         },
#     }
# }

In [3]:
# OPTION 3: Load from a folder structure
# Assumes: base_folder/drums/kick.wav, base_folder/bass/bass.wav, etc.

def load_from_folder_structure(base_path: str) -> Dict:
    """Load channels from organized folder structure"""
    channels = {}
    base = Path(base_path)
    
    # Expected categories - INCLUDES synths and backvocals
    categories = ['drums', 'bass', 'guitars', 'keys', 'vocals', 'backvocals', 'synths', 'strings', 'brass', 'percussion', 'fx', 'other']
    
    for category in categories:
        category_path = base / category
        if category_path.exists():
            channels[category] = {}
            for audio_file in category_path.glob('*.wav'):
                # Use filename (without extension) as channel name
                channel_name = audio_file.stem
                channels[category][channel_name] = str(audio_file)
    
    return channels

# Example usage:
channels = load_from_folder_structure("/Users/itay/Documents/post_mix_data/pre_mix_channels/")

# Display loaded channels
def display_channels(channels):
    print("📁 Loaded Channels:")
    total = 0
    for category, tracks in channels.items():
        if tracks:
            print(f"\n  {category.upper()}:")
            for name, path in tracks.items() if isinstance(tracks, dict) else []:
                if isinstance(path, dict):
                    print(f"    • {name}: {path['path']} {path.get('hints', '')}")
                else:
                    print(f"    • {name}: {path}")
                total += 1
    print(f"\n  Total: {total} channels")

# display_channels(channels)

## 🎚️ Step 2: Choose Mix Template

Select a genre-specific mixing template or create a custom one. This determines:
- Processing chains per instrument
- Spatial positioning
- Effect sends
- Bus routing
- Overall mix character

In [4]:
# Available mix templates
MIX_TEMPLATES = [
    "modern_pop",       # Bright, wide, polished
    "rock",            # Punchy drums, present guitars
    "edm",             # Huge bass, side-chained, wide
    "hip_hop",        # Knock, 808s, crispy highs
    "jazz",           # Natural, dynamic, spatial
    "classical",      # Natural, orchestral positioning
    "indie",          # Lo-fi character, centered
    "metal",          # Aggressive, scooped, tight
    "r&b",            # Smooth, warm, vocal-focused
    "country",        # Natural, Nashville-style
    "reggae",         # Dub-style, bass-heavy
    "funk",           # Groovy, mid-focused
    "custom"          # Build your own
]

# Select template
selected_template = "modern_pop"  # Change this to your genre

# Optional: Customize template parameters
template_customization = {
    "brightness": 0.7,      # 0-1 (dark to bright)
    "width": 0.8,          # 0-1 (mono to wide)
    "aggression": 0.5,     # 0-1 (gentle to aggressive)
    "vintage": 0.3,        # 0-1 (modern to vintage)
    "dynamics": 0.6,       # 0-1 (compressed to dynamic)
    "depth": 0.7,          # 0-1 (flat to deep)
}

print(f"🎛️ Selected Template: {selected_template}")
print("\n📊 Template Characteristics:")
for param, value in template_customization.items():
    bar = '█' * int(value * 10) + '░' * int((1-value) * 10)
    print(f"  {param:12} [{bar}] {value:.1%}")

🎛️ Selected Template: modern_pop

📊 Template Characteristics:
  brightness   [███████░░░] 70.0%
  width        [████████░] 80.0%
  aggression   [█████░░░░░] 50.0%
  vintage      [███░░░░░░░] 30.0%
  dynamics     [██████░░░░] 60.0%
  depth        [███████░░░] 70.0%


## 🔍 Step 3: Channel Analysis & Auto-Configuration

The system analyzes each channel to determine optimal processing:
- Frequency content analysis
- Transient detection
- Dynamic range measurement
- Suggested processing chain

In [5]:
# Initialize mixing session
session = MixingSession(
    channels=channels,
    template=selected_template,
    template_params=template_customization,
    sample_rate=44100,  # Will auto-detect from files
    bit_depth=24
)

# Analyze all channels
print("🔍 Analyzing Channels...\n")
channel_analysis = session.analyze_all_channels()

# Display analysis results
for category, tracks in channel_analysis.items():
    if tracks:
        print(f"\n📊 {category.upper()}:")
        for name, analysis in tracks.items():
            print(f"\n  {name}:")
            print(f"    • Detected Type: {analysis['detected_type']}")
            print(f"    • Frequency Range: {analysis['freq_range']}")
            print(f"    • Dynamic Range: {analysis['dynamic_range']:.1f} dB")
            print(f"    • Suggested Chain: {' → '.join(analysis['suggested_chain'])}")
            if analysis.get('warnings'):
                print(f"    ⚠️ Warnings: {', '.join(analysis['warnings'])}")

print("\n✅ Analysis complete! Ready to mix.")

📁 Loading audio channels...
  ✓ Loaded: drums.KSHMR Acoustic Snare 03 (B)
  ✓ Loaded: drums.KSHMR Acoustic Kick 14 - Hard
  ✓ Loaded: drums.KSHMR_Acoustic_Kick_01_Big
  ✓ Loaded: drums.drums_TOMS mid
  ✓ Loaded: drums.drums_TOMS high
  ✓ Loaded: drums.drums_HATS accoustic
  ✓ Loaded: drums.drums_SNARE
  ✓ Loaded: drums.snare
  ✓ Loaded: drums.Hi Hat
  ✓ Loaded: drums.KSHMR Acoustic Snare 13 (F)
  ✓ Loaded: drums.drums_TOMS low
  ✓ Loaded: drums.drums_HATS electric support
  ✓ Loaded: drums.drums_CRASH 4
  ✓ Loaded: drums.KSHMR_Acoustic_Closed_Hat_01_Clean
  ✓ Loaded: drums.Kick snare
  ✓ Loaded: drums.drums_CRASH 3
  ✓ Loaded: drums.drums_CRASH 2
  ✓ Loaded: drums.drums_CRASH 1
  ✓ Loaded: drums.drums_KICK
  ✓ Loaded: bass.BASS chorus synt low stab
  ✓ Loaded: bass.Bass Synthwave
  ✓ Loaded: bass.BASS atmosphiric dark
  ✓ Loaded: bass.BASS chorus synt
  ✓ Loaded: bass.Bass chorus_quantized
  ✓ Loaded: bass.BASS chorus synt arpg
  ✓ Loaded: bass.Bass verse Ziv
  ✓ Loaded: guitars.GTR di

KeyboardInterrupt: 

## 🎛️ Step 4: Configure Mix Settings

Fine-tune the mixing parameters before processing. You can:
- Adjust individual channel settings
- Set bus routing
- Configure effects sends
- Define automation

In [6]:
# Global mix settings
mix_settings = {
    # Bus configuration
    "buses": {
        "drum_bus": {"channels": ["drums.*"], "compression": 0.3, "glue": 0.4},
        "bass_bus": {"channels": ["bass.*"], "compression": 0.4, "saturation": 0.2},
        "vocal_bus": {"channels": ["vocals.*"], "compression": 0.3, "presence": 0.5},
        "instrument_bus": {"channels": ["guitars.*", "keys.*"], "width": 0.7},
    },
    
    # Effect sends
    "sends": {
        "reverb_hall": {
            "type": "hall",
            "size": 0.7,
            "decay": 2.5,
            "sends_from": {
                "vocals.lead": 0.15,
                "guitars.lead": 0.10,
                "keys.piano": 0.12,
            }
        },
        "delay_quarter": {
            "type": "delay",
            "time": "1/4",
            "feedback": 0.3,
            "sends_from": {
                "vocals.lead": 0.08,
                "guitars.lead": 0.06,
            }
        },
    },
    
    # Master bus
    "master": {
        "eq_mode": "gentle",      # gentle, surgical, vintage
        "compression": 0.2,        # 0-1
        "limiter": True,
        "target_lufs": -14,       # For streaming
    },
    
    # Automation (optional)
    "automation": {
        "vocal_rides": True,      # Auto-adjust vocal level
        "drum_fills": True,       # Boost drum fills
        "outro_fade": False,      # Auto fade-out
    }
}

# Apply settings to session
session.configure(mix_settings)

print("✅ Mix configured with:")
print(f"  • {len(mix_settings['buses'])} buses")
print(f"  • {len(mix_settings['sends'])} effect sends") 
print(f"  • Target: {mix_settings['master']['target_lufs']} LUFS")

✅ Mix configured with:
  • 4 buses
  • 2 effect sends
  • Target: -14 LUFS


In [7]:
# Optional: Override specific channel settings
channel_overrides = {
    "drums.kick": {
        "eq": {"60hz": +2, "4khz": +1.5},  # Boost lows and click
        "compression": {"ratio": 4, "attack": 10},
        "gate": True,
    },
    "bass.bass_di": {
        "eq": {"80hz": +1, "800hz": -2},
        "compression": {"ratio": 3, "attack": 30},
        "blend_with": "bass.bass_amp",  # Blend DI with amp
    },
    "vocals.lead": {
        "eq": {"200hz": -1, "3khz": +2, "10khz": +1},
        "compression": {"ratio": 3, "attack": 5},
        "de_esser": True,
        "auto_tune": {"strength": 0.3, "key": "Cmaj"},  # Optional tuning
    },
}

# Apply overrides
for channel, settings in channel_overrides.items():
    session.override_channel_settings(channel, settings)

print(f"✅ Applied {len(channel_overrides)} channel overrides")

✅ Applied 3 channel overrides


## 🎚️ Step 5: Process the Mix

Run the mixing engine to create your professional mix. The system will:
1. Process each channel with its configured chain
2. Apply spatial positioning
3. Sum to buses
4. Apply bus processing
5. Create final mix and stems

In [8]:
# Create output directory
from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = f"/Users/itay/Documents/post_mix_data/mixing_sessions/session_{timestamp}"
os.makedirs(output_dir, exist_ok=True)

print(f"📁 Output directory: {output_dir}")
print("\n🎛️ Starting mix processing...\n")

# Process the mix
mix_results = session.process_mix(
    output_dir=output_dir,
    export_individual_channels=False,  # Set True to export each processed channel
    export_buses=True,                 # Export bus mixes
    export_stems=True,                 # Export stems for mastering
    export_full_mix=True,              # Export full stereo mix
    progress_callback=lambda msg: print(f"  {msg}")
)

print("\n✅ Mix processing complete!")
print("\n📊 Mix Results:")
print(f"  • Peak Level: {mix_results['peak_db']:.1f} dBFS")
print(f"  • RMS Level: {mix_results['rms_db']:.1f} dBFS")
print(f"  • LUFS: {mix_results['lufs']:.1f}")
print(f"  • Dynamic Range: {mix_results['dynamic_range']:.1f} dB")
print(f"  • Processing Time: {mix_results['time']:.1f} seconds")

📁 Output directory: /Users/itay/Documents/post_mix_data/mixing_sessions/session_20250827_220522

🎛️ Starting mix processing...

  Processing individual channels...
  Processing drum_bus...
  Processing bass_bus...
  Processing vocal_bus...
  Processing instrument_bus...
  Creating full mix...


NameError: name 'sr' is not defined

## 📊 Step 6: Export Stems for Post-Mix Pipeline

Export the mixed stems in the format expected by your existing post-mix mastering pipeline.

In [ ]:
# Export stems for post-mix pipeline
stem_export_config = {
    "format": "wav",
    "bit_depth": 24,
    "sample_rate": 44100,
    "normalization": "peak",  # peak, lufs, or none
    "target_level": -6.0,     # dBFS for peak, LUFS for lufs
}

# Map our mix buses to post-mix stem format
stem_mapping = {
    "drums": ["drum_bus"],          # All drum channels → drums.wav
    "bass": ["bass_bus"],           # All bass channels → bass.wav
    "vocals": ["vocal_bus"],        # All vocal channels → vocals.wav
    "music": ["instrument_bus"],    # All other instruments → music.wav
}

print("📤 Exporting stems for post-mix pipeline...\n")

exported_stems = session.export_stems(
    output_dir=os.path.join(output_dir, "stems"),
    stem_mapping=stem_mapping,
    config=stem_export_config
)

print("✅ Stems exported:")
for stem_name, stem_path in exported_stems.items():
    size_mb = os.path.getsize(stem_path) / (1024 * 1024)
    print(f"  • {stem_name}: {stem_path} ({size_mb:.1f} MB)")

# Save stem paths for post-mix pipeline
stems_for_postmix = {
    "drums_path": exported_stems["drums"],
    "bass_path": exported_stems["bass"],
    "vocals_path": exported_stems["vocals"],
    "music_path": exported_stems["music"],
}

# Save to JSON for easy loading in post-mix notebook
stems_json_path = os.path.join(output_dir, "stems_for_postmix.json")
with open(stems_json_path, 'w') as f:
    json.dump(stems_for_postmix, f, indent=2)

print(f"\n📝 Stem paths saved to: {stems_json_path}")
print("\n🎯 Ready for post-mix mastering pipeline!")

## 🔄 Step 7: Integrate with Post-Mix Pipeline

Now that your stems are mixed, you can run them through your existing post-mix mastering pipeline with all its features!

In [ ]:
# Option 1: Direct integration
print("🔄 Launching post-mix pipeline with mixed stems...\n")

# Import the post-mix pipeline
from mastering_orchestrator import MasteringOrchestrator
from config import CONFIG

# Set stem mode
CONFIG.pipeline.default_mode = CONFIG.pipeline.STEM_MASTERING

# Load the mixed stems
stem_paths = stems_for_postmix

print("✅ Stems loaded into post-mix pipeline:")
for stem_type, path in stem_paths.items():
    print(f"  • {stem_type}: {os.path.basename(path)}")

print("\n🎯 You can now:")
print("  1. Run all 30+ stem variants (RadioReady, 3D_Immersive, etc.)")
print("  2. Apply mastering styles (Neutral, Warm, Bright, Loud)")
print("  3. Generate streaming previews")
print("  4. Create comparison reports")
print("\n📖 Continue with your regular post-mix workflow!")

In [ ]:
# Option 2: Save mix session for later use
session_data = {
    "timestamp": timestamp,
    "channels": channels,
    "template": selected_template,
    "settings": mix_settings,
    "results": mix_results,
    "stems": stems_for_postmix,
    "output_dir": output_dir,
}

session_file = os.path.join(output_dir, "mix_session.json")
with open(session_file, 'w') as f:
    json.dump(session_data, f, indent=2)

print(f"💾 Mix session saved: {session_file}")
print("\n📝 To load in post-mix notebook, use:")
print(f'stem_paths = json.load(open("{stems_json_path}"))')

## 📈 Optional: Mix Visualization & Analysis

In [ ]:
# Visualize the mix
import matplotlib.pyplot as plt

# Create mix visualization
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

# 1. Frequency spectrum
session.plot_spectrum(axes[0, 0])
axes[0, 0].set_title('Mix Frequency Spectrum')

# 2. Stereo field
session.plot_stereo_field(axes[0, 1])
axes[0, 1].set_title('Stereo Field')

# 3. Dynamic range
session.plot_dynamics(axes[1, 0])
axes[1, 0].set_title('Dynamic Range')

# 4. Level meters
session.plot_levels(axes[1, 1])
axes[1, 1].set_title('Level Meters')

plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'mix_analysis.png'))
plt.show()

print("📊 Mix analysis saved!")